In [6]:
Pkg.add("DistributedArrays")

INFO: Cloning cache of DistributedArrays from git://github.com/JuliaParallel/DistributedArrays.jl.git
INFO: Installing DistributedArrays v0.1.3
INFO: Package database updated


In [7]:
using DistributedArrays

In [18]:
u=distribute([1:16])


16-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16

 in depwarn at /Volumes/Julia/Julia-0.4.0-rc1.app/Contents/Resources/julia/lib/julia/sys.dylib
 in oldstyle_vcat_warning at /Volumes/Julia/Julia-0.4.0-rc1.app/Contents/Resources/julia/lib/julia/sys.dylib
 in vect at abstractarray.jl:32
 in include_string at loading.jl:233
 in execute_request_0x535c5df2 at /Users/alanedelman/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/alanedelman/.julia/v0.4/IJulia/src/IJulia.jl:130
 in anonymous at task.jl:448
while loading In[18], in expression starting on line 1


In [16]:
function star(f1, f2, f3, args...)
    out1 = [remotecall(p, () -> f1(map(localpart, args)...)) for p = workers()]
    out2 = f2(map(fetch, out1), args...)
    out3 = RemoteRef[]
    for i = 1:length(workers())
        p = workers()[i]
        out2i = out2[i]
        push!(out3, remotecall(p, () -> f3(out2i, map(localpart, args)...)))
    end
    return DArray(out3)
end

star (generic function with 1 method)

In [17]:
lag(x::Vector) = [0, x[2:end-1]]
f1 = t -> cumsum!(t,t)[end] 
f2 = (out, _) -> lag(cumsum(out))
f3 =  .+
star(f1,f2,f3,copy(u))

16-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
   1
   3
   6
  10
  15
  21
  28
  36
  45
  55
  66
  78
  91
 105
 120
 136

 in depwarn at /Volumes/Julia/Julia-0.4.0-rc1.app/Contents/Resources/julia/lib/julia/sys.dylib
 in oldstyle_vcat_warning at /Volumes/Julia/Julia-0.4.0-rc1.app/Contents/Resources/julia/lib/julia/sys.dylib
 in vect at abstractarray.jl:38
 in lag at In[17]:1
 in anonymous at In[17]:3
 in star at In[16]:3
 in include_string at loading.jl:233
 in execute_request_0x535c5df2 at /Users/alanedelman/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/alanedelman/.julia/v0.4/IJulia/src/IJulia.jl:130
 in anonymous at task.jl:448
while loading In[17], in expression starting on line 5


In [14]:
u

LoadError: LoadError: indexing not defined for DistributedArrays.DArray{Float64,1,Array{Float64,1}}
while loading In[14], in expression starting on line 1

In [1]:
1+1

2

In [ ]:
α